# Setup

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
working_directory = 'GitHub/dl-superresolution-ipynb'
%cd /content/drive/MyDrive/$working_directory
!git status

/content/drive/MyDrive/GitHub/dl-superresolution-ipynb
Refresh index: 100% (4/4), done.
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   dl-superresolution.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


# Git Management

In [ ]:
!git config --global user.email "e11909390@student.tuwien.ac.ar"

In [7]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   dl-superresolution.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [8]:
!git commit -a -m "added connection for google colab"

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@586961a6959d.(none)')
